# Spatial Lambda Fleming Viot process

Класс модели:
    - Аттрибуты:
        - Параметры модели
        - Вложенные классы:
            - Класс состояния:
                - Время
                - Список особей с их координатами в настоящий м
            

In [867]:
import numpy as np
import matplotlib.pyplot as plt
import scipy as sc
import scipy.integrate as integr
from tqdm import tqdm
from icecream import ic
from anytree import NodeMixin, RenderTree
from anytree import find_by_attr, PreOrderIter
import pandas as pd
%load_ext Cython

The Cython extension is already loaded. To reload it, use:
  %reload_ext Cython


In [868]:
def tor2_distance(x, y, w=1, h=1):
    return (min(abs(x[0] - x[1]), w - abs(x[0] - x[1]))**2 + min(abs(y[0] - y[1]), h - abs(y[0] - y[1]))^2)**0.5

In [974]:
import pandas as pd
import numpy as np
import scipy as sc
import scipy.integrate as integr
from tqdm import tqdm
from anytree import NodeMixin, RenderTree
from anytree import find_by_attr, PreOrderIter
import matplotlib.pyplot as plt
class Node(NodeMixin):
    def __init__(self, index=0, time=0, parent=None, children=None):
        self.id = index
        self.parent = parent
        self.time=time
        if children:  # set children only if given
             self.children = children
    def __repr__(self):
             return str(f'id:{self.id}, time:{self.time}')
    def __str__(self):
        return str(f'id:{self.id}, time:{self.time}')
    
    
class State:
    # cdef public double time
    # cdef public int freeId
    def __init__(self, n_alleles, time=0):
        self.time = 0
        self.n_alleles = n_alleles
        self.individuals= pd.DataFrame(columns=['Id', 'ParentId', 'time', 'x', 'y']+ list(range(1, n_alleles+1)))
        self.individuals.set_index('Id', inplace= True)
        self.hist = pd.DataFrame(columns=['Id', 'ParentId', 'time', 'x', 'y']+ list(range(1, n_alleles+1)))
        self.hist.set_index('Id', inplace=True)
        
        
    def create(self, individuals):
        self.individuals = pd.DataFrame(individuals, columns=list(self.individuals))
        self.hist = pd.DataFrame(individuals, columns=list(self.individuals))
        
        
    def delete(self, ids):
        self.individuals.drop(ids, inplace=True)
    
    
    def add(self, individuals):
        self.individuals = pd.concat([self.individuals, pd.DataFrame(individuals, columns=list(self.individuals))], ignore_index=True)
        self.hist = pd.concat([self.hist, pd.DataFrame(individuals, columns=list(self.individuals))], ignore_index=True)
        
    
    def genealogy(self, ids ):
        table = self.hist[self.hist[:,0] == ids[0]]
        for idx in ids[1:]:
            table = np.row_stack((table, self.hist[self.hist[:,0] == idx]))
        allids = set(ids)
        newids = list(set(table[:,1]) - allids)
        allids = allids.union(set(newids))
        while len(newids)>0:
            for idx in newids:
                table = np.row_stack((table, self.hist[self.hist[:,0] == idx]))
            newids = list(set(table[:,1]) - allids)
            allids = allids.union(set(newids))
        return table
        
    
    
    
    def _build_coalesce_tree(self, data, parent=None, index=0):
    # data = [[id, parent_id, time]]
        root = None
        if parent is None:
            root = Node()
            parent = root
        for x in data[data[:,1]==index]:
            node = Node(x[0],x[2],parent)
            self._build_coalesce_tree(data, node, x[0])
        return root
        
        
    def build_coalesce_tree(self, ids):
        data = self.genealogy(ids)
        return self._build_coalesce_tree(data)
            

class Model:
    # cdef public double rho, L, u0, theta, lamda
    # cdef public n_alleles
    
    
    
    def __init__(self, L=1., lamda=1., u0=0.4, rho=1000., theta=0.5, alpha=1., n_alleles=10):
        self.rho=rho
        self.L=L
        self.u0=u0
        self.theta=theta
        self.alpha=alpha
        self.lamda = lamda
        self.n_alleles = n_alleles
        self.dynamic=None
        self.state = State(n_alleles)
        
        
        
        
        
#--------------Initializating functions-------------------
        
        
    def generate_dynamic(self, n_epoch, time_init=0):
        '''Result is in (time, x, y)
        NOTE! MAYBE LAMBDA SHOULD BE RENORMALISED'''
        times = time_init + np.cumsum(np.random.exponential(self.lamda, n_epoch))
        xs = np.random.uniform(0, self.L, n_epoch)
        ys = np.random.uniform(0, self.L, n_epoch)
        self.dynamic = np.column_stack((xs, ys, times))
        
        
    def generate_initial_points(self):
        '''Fill state.individuals with points according to uniform poisson point process with density \\rho'''
        N_points = np.random.poisson(self.rho*self.L**2)
        xs = np.random.uniform(0, self.L, N_points)
        ys = np.random.uniform(0, self.L, N_points)
        return np.column_stack((xs, ys))
        
        
    def initiate(self, proport=0.5):
        N_points = np.random.poisson(self.rho*self.L**2)
        xs = np.random.uniform(0, self.L, N_points)
        ys = np.random.uniform(0, self.L, N_points)
        alleles = np.random.choice([0, 1], (N_points, self.n_alleles), p = [1 - proport, proport])
        pId = np.full(N_points, 0)
        times = np.full(N_points, 0)
        self.state.create(np.column_stack((pId, times, xs, ys ,alleles)))
        
    
    def choose_parent(self, z):
        probs = []
        for i in range(self.state.individuals.shape[0]):
            probs.append(self.v(z[0:2], self.state.individuals.iloc[i][['x','y']].values))
        return self.state.individuals.iloc[np.random.choice(np.arange(len(self.state.individuals)), p = probs/np.sum(probs))]
    
    
    def choose_parent_type(self, z):
        probs = []
        for i in range(self.state.individuals.shape[0]):
            probs.append(self.v(z[0:2], x[3:5])) 
        return self.state.individuals[np.random.choice(np.arange(len(self.state.individuals)), p = probs/np.sum(probs))][5:]
    
    
    
#--------------Evolution functions--------------------------    
    def extinction(self, event):
        z, time = event[0:2], event[2]
        time = 0
        # time = event[2]
        ids = [] #ids to delete
        for i in range(self.state.individuals.shape[0]):
            if np.random.uniform() < self.u(z, self.state.individuals.iloc[i][['x','y']].values):
                ids.append(i)
                # print(f'{indicies=}')
        # print(f'survived {points[indicies].shape=}')
        self.state.delete(ids)
    
    
    def recolonization(self, event):
        z, time = event[0:2], event[2]
        parent = self.choose_parent(z)
        parentId = parent.name
        parentType = parent[range(1, self.n_alleles+1)].values
        intensity = lambda x, y: self.u(z, np.array([x,y]))
        max_intensity = intensity(z[0], z[1])
        total_intensity = integr.dblquad(intensity, 0, self.L, 0,  self.L )[0]
        # print(f"{total_intensity=}\n{max_intensity=}")
        n_points = np.random.poisson(self.rho * total_intensity) # Тут вроде total
        # print(f'{rho * total_intensity=}')
        # print(f'recolonized {n_points=}')
        points = []
        generated = 0
        while generated < n_points:
            x = np.random.uniform(0, self.L)
            y = np.random.uniform(0, self.L)

            if self.L**2 * intensity(x,y) >= np.random.uniform(0, max_intensity):
                points.append([x,y])
                generated += 1
        
        
        points = np.array(points,ndmin=2)
        points = np.column_stack(
            (
                np.full(n_points,parentId),
                np.full(n_points,time),
                points,
                np.full((n_points, self.n_alleles), parentType)
            )
        )
        self.state.add(points)
   

    
    
    
    def propagate(self, event):# parameters -- list [L, rho, u0, alpha, theta]
        # ic(event)
        # pass
        self.extinction(event)
        self.recolonization(event)
    
    
    def run(self):
        for event in tqdm(self.dynamic):
            self.propagate(event) 
    
#----------------Hat functions------------------------    
    def v(self, z, x):
        return np.exp(- np.linalg.norm(z-x)/(2 * self.alpha**2 * self.theta**2))
    
    
    def u(self, z, x):
        return self.u0 * np.exp(- np.linalg.norm(z-x)/(2 * self.theta**2))
    
    
    def h(self, z, x, beta = 1):
        return np.exp(-np.linalg.norm(z-x)/beta**2)
    
    
#---------------ANALYS FUNCTIONS----------------------- 
    def density(self, z, beta=1):
        points = self.state.individuals
        denom = 0
        thetas = np.zeros(self.n_alleles)
        for x in points:
            denom += self.h(z, x[3:5], beta)
            for k in range(self.n_alleles):
                if x[k+5] == 1:
                    thetas[k] += self.h(z, x[3:5], beta)
        thetas = thetas / denom

        return thetas


    def plt_SFS1(self, z, beta=1):
        d = self.density(z, beta=1)
        N = 100
        y = []
        for i in range(N):
            y.append((d<(i+1)/100).sum())
        plt.plot(y)


    def plt_SFS2(self, z1, z2,beta=1):
        d1 = self.density(z1, beta=1)
        d2 = self.density(z2, beta=1)
        N = 100
        y = np.zeros((N,N))
        for i in range(N):
            for j in range(N):
                y[i,j]=(np.logical_and(d1<(i+1)/N, d2<(j+1)/N).sum())
        plt.imshow(y, extent=[0,1,0,1])
        
        
    def plot_with_alleles(self, allele=0, alpha=0.5):
        points = self.state.individuals
        plt.scatter(points[points[:,5+allele]==0][:,3],points[points[:,5+allele]==0][:,4], alpha, label ='0 allele')
        plt.scatter(points[points[:,5+allele]==1][:,3],points[points[:,5+allele]==1][:,4], alpha, label = '1 allele')
        plt.legend()
        plt.show();
#--------------------Auxilary
    
    def save(self):
        return (self.rho, 
                self.L,
                self.u0,
                self.theta,
                self.alpha,
                self.lamda,
                self.n_alleles,
                np.copy(self.dynamic),
                np.copy(self.state.individuals),
                np.copy(self.state.hist))

    
    def load(self, data):
        self.rho, 
        self.L,
        self.u0,
        self.theta,
        self.alpha,
        self.lamda,
        self.n_alleles,
        self.dynamic,
        self.state.individuals,
        self.state.hist = data
            

In [975]:
s = State(3)

In [976]:
list(s.individuals)

['ParentId', 'time', 'x', 'y', 1, 2, 3]

In [977]:
s.create(np.array([[2,3,4,5,6, 2, 3], [3,4,5,6, 7, 6, 3]]))

In [973]:
s.individuals[range(1,4)].values

array([[6, 2, 3],
       [7, 6, 3]])

In [866]:
s.individuals.drop([3,4])

,ParentId,time,x,y,1
0,2,3,4,5,6
1,3,4,5,6,7
2,2,3,4,5,6
5,3,4,5,6,7
6,2,3,4,5,6
7,3,4,5,6,7


In [916]:
s.individuals.iloc[1][['x','y']].values + np.array([1,2])

array([6, 8])

In [914]:
s.individuals.iloc[1].values

array([3, 4, 5, 6, 7])

In [988]:
a = Model(
    rho = 1000,
    L = 1,
    lamda = 1,
    u0 = 0.4,
    alpha = 1,
    theta = 0.3,
    n_alleles = 5
)

In [989]:
a.generate_dynamic(100)

In [990]:
a.initiate(0.4)

In [991]:
a.state.individuals

,ParentId,time,x,y,1,2,3,4,5
0,0.0,0.0,0.498587,0.537458,0.0,1.0,0.0,0.0,1.0
1,0.0,0.0,0.477317,0.982001,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.427283,0.068215,0.0,1.0,1.0,0.0,1.0
3,0.0,0.0,0.747975,0.314447,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.841444,0.872773,1.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...
993,0.0,0.0,0.993774,0.025488,0.0,0.0,1.0,0.0,1.0
994,0.0,0.0,0.124155,0.571713,1.0,1.0,0.0,0.0,1.0
995,0.0,0.0,0.783439,0.769096,1.0,0.0,0.0,0.0,0.0
996,0.0,0.0,0.707834,0.967953,1.0,0.0,0.0,0.0,1.0


In [985]:
a.recolonization(np.array([0.5,0.5, 2]))

In [994]:
%%time
a.run()

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 100/100 [00:44<00:00,  2.27it/s]

CPU times: user 43.7 s, sys: 1.93 s, total: 45.7 s
Wall time: 44.1 s


In [720]:
%%cython

cpdef int add(int x, int y):
    return (x+y)

ld: warning: -pie being ignored. It is only used when linking a main executable


In [724]:
add(1,5.3)

6

In [685]:
a.state.hist.shape

(1971, 10)

In [686]:
a.state.genealogy([1800, 1600, 1400])

array([[1.80000000e+03, 1.71600000e+03, 2.30515006e+01, 6.12801701e-01,
        1.24166558e-01, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        1.00000000e+00, 1.00000000e+00],
       [1.60000000e+03, 1.84000000e+02, 1.57332431e+01, 7.45795052e-01,
        6.27466489e-01, 1.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        1.00000000e+00, 1.00000000e+00],
       [1.40000000e+03, 1.24000000e+02, 1.32439410e+01, 8.73157967e-01,
        6.52011640e-01, 0.00000000e+00, 0.00000000e+00, 1.00000000e+00,
        1.00000000e+00, 1.00000000e+00],
       [1.84000000e+02, 0.00000000e+00, 0.00000000e+00, 8.86388945e-02,
        6.73335790e-01, 1.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        1.00000000e+00, 1.00000000e+00],
       [1.71600000e+03, 3.25000000e+02, 1.72933332e+01, 3.22593021e-01,
        6.56269861e-01, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        1.00000000e+00, 1.00000000e+00],
       [1.24000000e+02, 0.00000000e+00, 0.00000000e+00, 6.60923233e-01,
   

In [687]:
print(RenderTree(a.state.build_coalesce_tree([1800, 1600, 1400])))

id:0, time:0
├── id:184.0, time:0.0
│   └── id:1600.0, time:15.733243086635982
├── id:124.0, time:0.0
│   └── id:1400.0, time:13.24394102016084
└── id:325.0, time:0.0
    └── id:1716.0, time:17.293333211584905
        └── id:1800.0, time:23.051500567529736


In [ ]:
def build_coalesce_tree(data, parent=None, index=0):
# data = [[id, parent_id, time]]
    root = None
    if parent is None:
        root = Node()
        parent = root
    for x in data[data[:,1]==index]:
        node = Node(x[0],x[2],parent)
        build_coalesce_tree(data, node, x[0])
    return root

In [893]:
s.individuals.iloc[1][['x','y']].to_numpy()

array([5, 6])

In [900]:
s.individuals.drop(2).iloc[[1,2,3]].index

1

In [ ]:
s.individuals

In [903]:
for x in s.individuals.to_numpy():
    print(x)

[2 3 4 5 6]
[3 4 5 6 7]
[2 3 4 5 6]
[3 4 5 6 7]
[2 3 4 5 6]
[3 4 5 6 7]
[2 3 4 5 6]
[3 4 5 6 7]
